# 使用GPU

## 1. 单GPU环境
1. 默认情况下，tensor是在cpu上创建的，可以用tensor.to()将tensor转移到gpu上，或者直接在gpu上新建tensor
2. 一个expression中的运算对象要在同一个device上，cpu和gpu上的数据无法在一个expression中处理
3. cpu上的tensor可以跟numpy共享底层的memory location，但gpu上的tensor不行。因为gpu上只能放tensor，不能放numpy数据类型
4. 使用device handle，不要直接用device name string来设置tensor的位置。前者更容易在不同的设备环境上运行代码，后者一旦改变硬件环境，当name string变化时，代码就不可用
5. 要注意，在cpu和gpu之间做数据的迁移很耗费时间。

In [1]:
import torch
import numpy as np

In [17]:
## 在gpu上新建tensor
x = torch.rand(2, 2, device='cuda')
print(x)

## 将现有tensor移动到gpu
x = torch.ones(2, 3)
if torch.cuda.is_available():
    y = x.to('cuda')
print(x.shape, x.dtype, x.device)
print(y.shape, y.dtype, y.device)

# mul = tensor * tensor2 # 错，他们一个在cpu上，一个在gpu上

tensor([[0.0024, 0.6778],
        [0.2441, 0.6812]], device='cuda:0')
torch.Size([2, 3]) torch.float32 cpu
torch.Size([2, 3]) torch.float32 cuda:0


In [18]:
## cpu上的tensor可以跟numpy共享memory location。改变其中一个的值，另一个也变
t = torch.ones(5)
n = t.numpy()
print("改变前：", t, n)
t.add_(2)
print("改变tensor，numpy对象也变：", t, n)
np.add(n, 2, out=n)
print("改变numpy对象后，tensor也变：", t, n)

## 但在gpu上无法做numpy类型对象的计算
if torch.cuda.is_available():
    t = t.to('cuda')
#     n = n.to('cuda') # numpy对象不能移到gpu上，因为没有.to method
print(t)
# n = t.numpy() # 报错：can't convert cuda:0 device type tensor to numpy.

改变前： tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
改变tensor，numpy对象也变： tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]
改变numpy对象后，tensor也变： tensor([5., 5., 5., 5., 5.]) [5. 5. 5. 5. 5.]
tensor([5., 5., 5., 5., 5.], device='cuda:0')


In [19]:
## device handle
if torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')
print('Device: {}'.format(my_device))

x = torch.rand(2, 2, device=my_device)
print(x)

Device: cuda
tensor([[0.6815, 0.0556],
        [0.0711, 0.4825]], device='cuda:0')


## 2. 多GPU环境

In [16]:
## 查询gpu数量
torch.cuda.device_count()

## 如果gpu不止1个，可以指定tensor所在的gpu
device0, device1 = 'cuda:0', 'cuda:1'
#  移动
x = torch.ones(2, 3)
x = x.to(device1)
#  直接新建
x = torch.rand(2, 3, device=device1)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
